Use this jupyterkernel:
``C:\Users\84551\AppData\Roaming\jupyter\kernels\engibench``

In [1]:
from engibench.problems.power_electronics.source import power_electronics_v0

# Create an empty problem
problem = power_electronics_v0.build()

init: source_dir = d:\git_repos\EngiBench\engibench\problems\power_electronics\source
init: ngSpice64 dir = d:\git_repos\EngiBench\engibench\problems\power_electronics\ngSpice64\bin\ngspice.exe


In [2]:
original_netlist_path = "engibench/problems/power_electronics/data/netlist/5_4_3_6_10-dcdc_converter_1.net"  # sweep 141
bucket_id = "5_4_3_6_10"
problem.load_netlist(original_netlist_path, bucket_id)

In [ ]:
sweep_data = {
    "C_val": [0.000015485, 0.00001948, 0.000015185, 0.000002442, 0.000009287, 0.000015377],
    "L_val": [0.000354659, 0.000706596, 0.000195361],
    "T1_val": [0.867615857, 0.867615857, 0.867615857, 0.867615857, 0.867615857],
    "T2_val": [1, 1, 1, 1, 1],
    "L1_val": [1, 1, 0, 0, 1],
    "L2_val": [1, 1, 1, 0, 0],
}


sweep_data = {
    "C_val": [0.000015600, 0.00001948, 0.000015185, 0.000002442, 0.000009287, 0.000015377],
    "L_val": [0.000354659, 0.000706596, 0.000195361],
    "T1_val": [0.867615857, 0.867615857, 0.867615857, 0.867615857, 0.867615857],
    "T2_val": [1, 1, 1, 1, 1],
    "L1_val": [1, 0, 0, 1, 1],  # "L1_val": [1, 1, 0, 0, 1], "L2_val": [1, 1, 1, 0, 0]
    "L2_val": [1, 0, 1, 1, 0],
}

problem.simulate(design_variable=sweep_data)
print(problem.simulation_results)

Processing topology from original netlist path: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\5_4_3_6_10-dcdc_converter_1.net
rewriting netlist to: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\rewrite_control_5_4_3_6_10-dcdc_converter_1.net
Running command: ['d:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\ngSpice64\\bin\\ngspice.exe', '-o', 'd:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\data\\log_file\\5_4_3_6_10-dcdc_converter_1.log', 'd:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\data\\netlist\\rewrite_control_5_4_3_6_10-dcdc_converter_1.net']
Error report from _calculate_efficiency: 0
[0.01244983 0.9094711  0.74045004]


To reproduce the previous bug caused by saving repeated `@R0[i]` and using `np.dtype()`:

In [ ]:
import numpy as np

names = ["time", "time", "time"]
formats = [np.float64] * 3
rowdtype = np.dtype({"names": names, "formats": formats})

ValueError: name already used as a name or title

Create sweep data matrix for dataset.

In [ ]:
# Range for C values: [1e-6, 2e-5]
# Range for L values: [1e-6, 1e-3]
# Range for T1 values: [0.1, 0.9]
# Keep the binary values the same for now.

import numpy as np

sweep_data = {
    "C_val": [0.000015485, 0.00001948, 0.000015185, 0.000002442, 0.000009287, 0.000015377],
    "L_val": [0.000354659, 0.000706596, 0.000195361],
    "T1_val": [0.867615857, 0.867615857, 0.867615857, 0.867615857, 0.867615857],
    "T2_val": [1, 1, 1, 1, 1],  # constant in all scenarios
    "L1_val": [1, 1, 0, 0, 1],  # fixed
    "L2_val": [1, 1, 1, 0, 0],  # fixed
}


n = 2  # number of iterpolation in each dimension

count = 0
count_stop = 1000000000
with (
    open("./engibench/problems/power_electronics/data/dataset_V0_positive.csv", "w") as pos_file,
    open("./engibench/problems/power_electronics/data/dataset_V0_negative.csv", "w") as neg_file,
):
    header = "C1, C2, C3, C4, C5, C6, L1, L2, L3, T1, g, r, e\n"
    pos_file.write(header)
    neg_file.write(header)

    for C1 in np.linspace(1e-6, 2e-5, n):
        for C2 in np.linspace(1e-6, 2e-5, n):
            for C3 in np.linspace(1e-6, 2e-5, n):
                for C4 in np.linspace(1e-6, 2e-5, n):
                    for C5 in np.linspace(1e-6, 2e-5, n):
                        for C6 in np.linspace(1e-6, 2e-5, n):
                            for L1 in np.linspace(1e-6, 1e-3, n):
                                for L2 in np.linspace(1e-6, 1e-3, n):
                                    for L3 in np.linspace(1e-6, 1e-3, n):
                                        for T1 in np.linspace(0.1, 0.9, 9):  # this primarily affects g DcGain
                                            print("count", count, flush=True)
                                            count += 1
                                            sweep_data = {
                                                "C_val": [C1, C2, C3, C4, C5, C6],
                                                "L_val": [L1, L2, L3],
                                                "T1_val": [T1, T1, T1, T1, T1],
                                                "T2_val": [1, 1, 1, 1, 1],
                                                "L1_val": [1, 0, 0, 1, 1],
                                                "L2_val": [1, 0, 1, 1, 0],
                                            }
                                            problem.simulate(design_variable=sweep_data)
                                            g, r, e = problem.simulation_results
                                            g, r = np.abs(g), np.abs(r)
                                            line = f"{C1}, {C2}, {C3}, {C4}, {C5}, {C6}, {L1}, {L2}, {L3}, {T1}, {g}, {r}, {e}\n"

                                            if g is np.nan:
                                                neg_file.write(line)
                                            else:
                                                pos_file.write(line)

                                            if count >= count_stop:
                                                break

count 0
Processing topology from original netlist path: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\5_4_3_6_10-dcdc_converter_1.net
rewriting netlist to: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\rewrite_control_5_4_3_6_10-dcdc_converter_1.net
Running command: ['d:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\ngSpice64\\bin\\ngspice.exe', '-o', 'd:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\data\\log_file\\5_4_3_6_10-dcdc_converter_1.log', 'd:\\git_repos\\EngiBench\\engibench\\problems\\power_electronics\\data\\netlist\\rewrite_control_5_4_3_6_10-dcdc_converter_1.net']
Error report from _calculate_efficiency: 1
count 1
Processing topology from original netlist path: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\5_4_3_6_10-dcdc_converter_1.net
rewriting netlist to: d:\git_repos\EngiBench\engibench\problems\power_electronics\data\netlist\rewrite_control_5_4_3_6_10-